# <span id="chap_parallel_client"></span> Using a remote client

In the [previous example](parallel-simple.ipynb) we placed all the components of the computational architecture on the same machine. This is the simplest set-up we can imagine, and is adequate for many situations. However, in practice we often want something more complicated, and in the next couple of chapters we'll deal with some some of these.

It used to be common for researchers to have powerful machines on their desks, but perhaps surprisingly it's becoming less common as people use laptops for more and more work. There may still be a machine in the lab, but it's often not used interactively and it might be better to keep the interactive parts of a system &ndash; the notebook &ndash; on a laptop with the processor-intensive computing parts sitting on the lab server. In this chapter we'll deal with connecting a client on one machine to a compute cluster on another. [Later](parallel-async.ipynb) we'll look at an even more attractive option, where we can disconnect from the cluster and check-back later (from home? from the beach?) to see how a long-running computation is getting on. 

## <span id="sec_parallel_client_remote_machine"></span> Find another machine

To make use of a remote client, we of course need a machine that isn't the one we'll be running the notebook on. Find a spare machine: ideally a powerful one that we can run a load of engines on. We'll refer to this as the "remote" machine, with the machine running the notebooks being the "local" machine. What we're looking for is a set-up like this:

<div class=figure id=fig_ipython_remote_client_cluster>
<div class=figurebody>
<img alt="A cluster with a remote client" src="ipython-remote-client-parallelism.svg">
<br>
<span class=caption>A simple compute cluster with a remote client.</span>
</div>
</div>

You'll need to be able to log-in to the remote machine, which we assume you'll do with `ssh`, the Secure Shell, which is the standard tool that lets you open a command-line shell on a remote machine and issue commands. `ssh` is a wonderfully complicated piece of kit that can be used to secure all sorts of interactions over the internet. It's most often used interactively to log in, but it can also be used automatically by programs, and IPython uses it to secure interactions between controller and engines.

You should be using `ssh` without passwords, since we want to automate the whole system and not be entering passwords all the time. If this is already set up for you, then you can skip to [setting up a profile](#sec_parallel_client_profile); if not, read on.

## <span id="sec_parallel_client_ssh_keypairs"></span> The proper use of `ssh` &ndash; 1

How can we use `ssh` to communicate between programs, without having to enter passwords? The answer is the *keypair*.

As the name suggests, a keypair consists of two "keys", which are cryptographic tokens that you use to identify yourself. A keypair consists of a *private* key that you keep somewhere safe and a *public* key that can be made freely available, sent over the internet, or even placed on an open web server: they really are *that* public. The details of how keypairs work need not concern us &ndash; there's a good [Wikipedia article](https://en.wikipedia.org/wiki/Public-key_cryptography) if you're interested &ndash; but the important point is that you place your private key on any machine you log-in *from*, your public key on any machine you log-in *to*, and `ssh` will be able to authenticate you and secure your interactions against snooping without you having to do anything else. More importantly, programs that are `ssh`-aware &ndash; as IPython is &ndash; will also be able to interact securely and automatically through a process called "`ssh` tunneling" whereby they route all their communications through a secure `ssh`-protected connection. 

To create a keypair you use the `ssh-keygen` tool:

In [3]:
%%bash

ssh-keygen -t rsa -b 2048 -N '' -f sample_rsa

Generating public/private rsa key pair.
Your identification has been saved in sample_rsa.
Your public key has been saved in sample_rsa.pub.
The key fingerprint is:
SHA256:2G689qdj7qYwf1nN3gAhqarX/GtwJOaCMqc5IUVqXvQ sd@functor.home
The key's randomart image is:
+---[RSA 2048]----+
|           .     |
|  ..      o .    |
| o. .    . . .   |
|.... E o+ . .    |
|o..  ..+So   +   |
|..+ o ooo . . +  |
| . B .o++o o . o |
|  + . .=+.B . . .|
|   . . .+@B=     |
+----[SHA256]-----+


This creates a keypair as two files called `sample_rsa` and `sample_rsa.pub`, the private and public keys respectively. (You can use any name you like instead of `sample`.) Look inside them and you'll see some descriptive comments and a block of random-looking characters that are the machine representation of the keys. If you miss off the `-f` option, `ssh_keygen` generates a keypair called `id_rsa` and `id_rsa.pub` by default.

To use your keypair, you place both `sample_rsa` and `sample_rsa.pub` into the directory `~/.ssh` that should exist on the machine you want to log-in to places *from* &ndash; the local machine, your laptop, for example. You then go to the `~/.ssh` directory on the machines you want to log-in *to* &ndash; the remote machine, the compute server &ndash; and copy `sample_rsa.pub` there, using `scp`:

In [ ]:
%%bash

scp sample_rsa.pub john@my-remote-machine:/home/john/.ssh 

(assuming you have your cluster on a machine called `my-remote-machine` on which your username is `john` and `/home/john` is your home directory). You'll be asked for your `ssh` password to complete this operation. (As you might have guessed, `scp` is a version of the copy program `cp` that uses `ssh` to transfer files between machines.)

You then need to let `ssh` know that this public key can be used to log-in to `cluster`. Log-in using your password, change to the `~/.ssh` directory, and append the public key to the file `authorized_keys` that should already exist there:

In [ ]:
%%baah

cat sample_rsa.pub >>authorized_keys

Now log-out and try to `ssh` back into `my-remote-machine`. You should be allowed straight in without a password, with `ssh` using the keypair to automate your log-in.

There are a few more things we can do with `ssh`, but we'll defer that until we talk about running more complex compute clusters.

## <span id="sec_parallel_client_profile"></span> Running in a profile

The [simple cluster](parallel-simple.ipynb) lives up to its name. In fact it's a little *too* simple, and we'll first take a little more control over how things are set up. This will be important preparation both for remote clients and for [more complex cluster configurations](parallel-complex.ipynb).

All IPython clusters include a set of configuration files called a *profile*. Even local clusters have them, the profile being called `default`. Profiles are stored in user home directories in a directory called `~/.ipython`. Within this directory will be a directory `profile_default/` storing the configuration of the default profile.

Let's not concern ourselves immediately with what's in a profile, but simply create an empty one:

In [2]:
%%bash

ipython profile create sample --parallel

[ProfileCreate] Generating default config file: u'/Users/sd/.ipython/profile_sample/ipython_config.py'
[ProfileCreate] Generating default config file: u'/Users/sd/.ipython/profile_sample/ipython_kernel_config.py'
[ProfileCreate] Generating default config file: u'/Users/sd/.ipython/profile_sample/ipcontroller_config.py'
[ProfileCreate] Generating default config file: u'/Users/sd/.ipython/profile_sample/ipengine_config.py'
[ProfileCreate] Generating default config file: u'/Users/sd/.ipython/profile_sample/ipcluster_config.py'


This command creates a profile called `sample` that we'll use for exploratory purposes. If you look in the `~/.ipython/profile_sample` directory you'll find a collection of files and sub-directories. These files configure how IPython works when running "in the sample profile". You can have lots of different profiles, each of which can configure IPython slightly differently. We'll return to this [later](parallel-complex.ipynb).

For the time being, what we want to do is configure the profile to allow our cluster to be accessed remotely. We do this by editing the file `~/.ipython/profile_sample/ipcontroller_config.py`. If you look at this file you'll see a lot of Python code &ndash; all of which is commented-out, meaning that the profile uses the default options for everything. To configure the profile we either uncomment appropriate lines or add them to the end of the file: personally I think the latter makes life simpler.

We want to change several things. We need to tell the profile where the compute cluster lives: `my-remote-machine.example.com`, let's say. (Replace this with the actual name of your remote machine, obviously.)  We'll also set the system to re-use security information, for reasons that will become clear later. This leads to adding the following four lines to the end of the profile configuration:

<pre>
c.IPControllerApp.ssh_server = "my-remote-machine.example.com"
c.IPControllerApp.reuse_files = True
c.IPControllerApp.location"my-remote-machine.example.com"
c.HubFactory.ip = u'*'
</pre>

(We'll come back to the `location` and `ip` lines a little later, but it doesn't hurt to add them now.) That's all we need to do, we should have a workable profile set up in which to run our compute cluster.

## <span id="sec_parallel_client_start_cluster"></span> Starting the cluster

We can now start the cluster on `my-remote-machine` in almost exactly same way as we did for the local cluster we uesed [earlier](#parallel-simple.iypnb#sec_parallel_simple_ipcluster). The only difference is that we're going to run this cluster in the profile we just created:

In [ ]:
%%bash

ipcluster --profile=sample &

(Replace `sample` with whatever name you  used for the profile, if you used a different one.) This command will once again spin-up the default number of compute engines and connect them to a controller.

Take a look in the `~/.ipython/profile_sample/security` directory (again changing `profile_sample` to match the name of your profile). There are a couple of files in here, and the one we're interested in is called `ipcontroller-client.json`. If you open this with an editor, you'll find a [JSON](http://www.json.org/) object that defines the connection properties we need in order to connect to the cluster running in this profile. (The JSON object contains about a dozen fields, none of which need concern us at the moment.) In order to connect to the clietsr and run code on it, the client software &ndash; the IPython notebook or script that submits jobs to the cluster &ndash; needs access to this file. We therefore copy the file to our local client machine, into the same directoty where we keep the notebook:

In [ ]:
%%bash

scp john@my-remote-machine:/home/john/.ipython/profile_sample/security/ipcontroller-client.json .

## <span id="sec_parallel_client_connect_client"></span> Connecting to the cluster

We can now &ndash; finally! &ndash; connect to our remote cluster. To do this we simply need to tell IPython that it should be using the remote cluster &ndash; and we have all the information needed to do this in the client security file we copied over. The `Client` object controls the connection, and we can use exactly the same code structure as we used [before](parallel-simple.ipynb#sec_parallel_simple_programming), just passing the necessary connection information:

In [8]:
from ipyparallel import Client
import math

client = Client(url_file = 'ipcontroller-client.json',
                paramiko = True,
                sshkey = "/Users/sd/.ssh/identity")
with client[:].sync_imports():
    import numpy
    import math
view = client.load_balanced_view()

importing numpy on engine(s)
importing math on engine(s)


This should (all being well) connect the local client notebook to the remote compute cluster, to which we can then send jobs:


In [10]:
ns = [ 2, 6, 10, 20, 30, 40 ]
view.map_sync(math.factorial, ns)

[2,
 720,
 3628800,
 2432902008176640000,
 265252859812191058636308480000000L,
 815915283247897734345611269596115894272000000000L]

(We've used the `math.factorial` standard function instead of defining our own as we did before.)

So we can now connect a client (a notebook or a script) on one machine to a compute server running remotely, tunnelling everything over `ssh` to keep things secure. This can make life considerable simpler, and especially since we can [extend this technique](parallel-async.ipynb) to allow the client machine to be disconnected from on-going simulations.